This notebook is intended for exploring processed data

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math

from esdglider import gcp, glider, utils, plots

# deployment_info = {
#     "deployment_name": "calanus-20241019",
#     "mode": "delayed",
# }
# deployment_info = {
#     "deployment_name": "amlr03-20231128",
#     "mode": "delayed",
# }
deployment_info = {
    "deployment_name": "amlr08-20220513",
    "mode": "delayed",
}

# Standard 
deployment_name = deployment_info["deployment_name"]
mode = deployment_info["mode"]
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket("amlr-gliders-deployments-dev", deployments_path, ro=False)
deployment_info["deploymentyaml"] = os.path.join(
    config_path, 
    f"{deployment_info["deployment_name"]}.yml", 
)

paths = glider.get_path_deployment(deployment_info, deployments_path)

# Timeseries file exploration

In [ ]:
ds_raw = xr.load_dataset(paths["tsrawpath"])
df_raw = ds_raw.to_pandas()
display(ds_raw)

In [ ]:
ds_eng = xr.load_dataset(paths["tsengpath"])
df_eng = ds_eng.to_pandas()
display(ds_eng)

ds_sci = xr.load_dataset(paths["tsscipath"])
df_sci = ds_sci.to_pandas()
display(ds_sci)

Test interp find_gaps

In [ ]:
print(np.count_nonzero(np.isnan(ds_sci.oxygen_concentration)))
print(np.count_nonzero(np.isnan(ds_sci.conductivity)))
print(np.count_nonzero(np.isnan(ds_sci.chlorophyll)))

testing interp

In [ ]:
x = ds_sci.chlorophyll
display(x)
y = ds_raw.chlorophyll
display(y)
y_nona = y.dropna("time")
display(y_nona)

In [ ]:
y_interp = np.interp(
    x.time.values.astype(np.int64), 
    y_nona.time.values.astype(np.int64), 
    y_nona.values, 
    left=np.nan, 
    right=np.nan, 
)
y_interp

In [ ]:
pd.DataFrame(y_interp - x.values).describe()

In [ ]:
x.time.values[np.argwhere(np.isnan(x.values))]

In [ ]:
import pyglider.utils as pgutils
np.count_nonzero(pgutils.find_gaps(
    y_nona.time.values.astype(np.int64), 
    x.time.values.astype(np.int64), 
    60
))

old stuff

In [ ]:
oxy.time.max()

In [ ]:
oxysat = ds_raw["oxygen_saturation"].dropna("time").to_pandas()
oxysat

In [ ]:
oxy.oxygen_saturation.to_pandas().describe()

In [ ]:
import glidertools as gt 
# gt.plot.scatter(ds_sci.time, ds_sci.depth, ds_sci.oxygen_concentration)
ax1 = plots.sci_gt_plot(ds_sci, "oxygen_saturation", show = True)
ax2 = plots.sci_gt_plot(ds_sci, "oxygen_concentration", show = True)

In [ ]:
plots.sci_timeseries_loop(ds_sci, paths["plotdir"])

In [ ]:
import matplotlib.pyplot as plt
x = ds_sci.time.values
y = ds_sci.oxygen_saturation.values

plt.figure(figsize=(6, 4))
plt.scatter(x, y, color='blue', marker='o')

# Add labels and title
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.title('Basic Scatter Plot')

# Gridded file exploration

In [ ]:
path_g5 = os.path.join( paths["griddir"], f"{deployment_name}_grid-{mode}-5m.nc")
ds_g5 = xr.load_dataset(path_g5)
ds_g5

In [ ]:
ds_g5.profile_time_start.values

Chunk with which to run make_gridfiles in debug mdoe

In [ ]:
from pyglider import ncprocess as pgncprocess
pgncprocess.make_gridfiles(
    path_sci,
    "/home/sam_woodman_noaa_gov/grid-test",
    paths["deploymentyaml"],
    dz=5,
    fnamesuffix=f"-{mode}-5m",
)